In [12]:
import pandas as pd
import numpy as np

In [13]:
folder = 'G:\\transparency-in-pricing\\input_files\\'

file = '451502252_texas-health-harris-methodist-hospital-alliance_standardcharges.csv'

In [14]:
df = pd.read_csv(folder + file, dtype=str, skiprows=2, encoding='ansi')

In [15]:
df.rename(columns={
    'GenlDescr': 'description',
    'Package Type': 'contracting_method',
    'Code Type': 'line_type',
    'RevCode': 'rev_code',
    'Code': 'code',
    'NDC': 'ndc',
    'Charge Quantity': 'quantity'
}, inplace=True)

In [18]:
df.columns.tolist()

['description',
 'contracting_method',
 'line_type',
 'rev_code',
 'code',
 'ndc',
 'quantity',
 'Price',
 'Min IP TR',
 'Max IP TR',
 'Min IP',
 'Max IP',
 'Min OP',
 'Max OP',
 'Aetna ASA IP',
 'Aetna ASA OP',
 'Aetna Coventry IP TR',
 'Aetna Coventry IP',
 'Aetna Coventry OP',
 'BCBS Traditional IP',
 'BCBS Traditional OP',
 'ChoiceCare POB All Payer IP TR',
 'ChoiceCare POB All Payer IP',
 'ChoiceCare POB All Payer OP',
 'ChoiceCare PPO All Payer IP TR',
 'ChoiceCare PPO All Payer IP',
 'ChoiceCare PPO All Payer OP',
 'Cigna Indemnity IP',
 'Cigna Indemnity OP',
 'City of Fort Worth IP',
 'City of Fort Worth OP',
 'Galaxy IP TR',
 'Galaxy IP',
 'Galaxy OP',
 'Multiplan IP',
 'Multiplan OP',
 'Self Pay IP',
 'Self Pay OP',
 'USA MCO IP TR',
 'USA MCO IP',
 'USA MCO OP',
 'WebTPA IP',
 'WebTPA OP',
 'FWFF IP TR',
 'FWFF IP',
 'FWFF OP',
 'UHC All Payer IP',
 'UHC All Payer IP TR',
 'UHC All Payer OP',
 'UHC Options PPO IP',
 'UHC Options PPO IP TR',
 'UHC Options PPO OP',
 'Aetna Com

In [76]:
cols = df.columns.tolist()
id_vars = cols[:7]
value_vars = cols[7:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [77]:
df.dropna(subset='standard_charge', inplace=True)

In [78]:
df.loc[df['code'].str.match(r'^[A-Z][0-9]{4}$|^[0-9]{5}$|^[0-9]{4}[A-Z]$'), 'hcpcs_cpt'] = df['code']

In [79]:
df.loc[~df['rev_code'].isna(), 'rev_code'] = df['rev_code'].str.zfill(4)

In [80]:
df.loc[df['line_type'] == 'DRG', 'ms_drg'] = df['code']

In [81]:
df.loc[df['line_type'] == 'APR DRG', 'apr_drg'] = df['code']

In [82]:
df['apr_drg'] = df['apr_drg'].str[:3] + '-' + df['apr_drg'].str[-1]

In [83]:
df.loc[df['code'].str.match('NOCPT|No CPT'), 'code'] = pd.NA

In [84]:
df['contracting_method'] = df['contracting_method'].str.lower()
df['line_type'] = df['line_type'].str.lower()

In [85]:
payer_mapping = {
    'Price': 'gross',
    'Min IP TR': 'min',
    'Max IP TR': 'max',
    'Min IP': 'min',
    'Max IP': 'max',
    'Min OP': 'min',
    'Max OP': 'max'
}

df['payer_category'] = df['payer_name'].map(payer_mapping).fillna('payer')

In [86]:
df['setting'] = np.where(df['payer_name'].str.contains(' IP'), 'inpatient', np.where(df['payer_name'].str.contains(' OP'), 'outpatient', 1))

In [101]:
id_mapping ={
 '750972805_texas-health-arlington-memorial-hospital_standardcharges.csv': '450064',
 '481281376_texas-health-center-for-diagnostics-and-surgery-plano_standardcharges.csv': '450891',
 '756001743_texas-health-harris-methodist-hospital-fort-worth_standardcharges.csv': '450135',
 '451502252_texas-health-harris-methodist-hospital-alliance_standardcharges.csv': '670085',
 '751748586_texas-health-harris-methodist-hospital-azle_standardcharges.csv': '450419',
 '751977850_texas-health-harris-methodist-hospital-cleburne_standardcharges.csv': '450148',
 '020555370_texas-health-harris-methodist-hospital-southlake_standardcharges.csv': '450888',
 '752678857_texas-health-harris-methodist-hospital-southwest-fort-worth_standardcharges.csv': '450779',
 '751752253_texas-health-harris-methodist-hospital-stephenville_standardcharges.csv': '450351',
 '203003947_texas-health-heart-and-vascular-hospital-arlington_standardcharges.csv': '670071',
 '831954982_texas-health-hospital-frisco_standardcharges.csv': '670260',
 '260684968_texas-health-presbyterian-hospital-flower-mound_standardcharges.csv': '670068',
 '752771437_texas-health-presbyterian-hospital-kaufman_standardcharges.csv': '450292',
 '202848116_texas-health-hospital-rockwall_standardcharges.csv': '670044'}

hosp_id = id_mapping[file]

df['hospital_id'] = hosp_id

output_file = hosp_id + '_' + file.split('_')[1].replace('texas-health-', '') + '.csv'

df.to_csv('.\\output_files\\' + output_file, index=False)

In [107]:
df.loc[df['code'].str.len() < 3]

,description,contracting_method,line_type,rev_code,code,ndc,quantity,payer_name,standard_charge,hcpcs_cpt,ms_drg,apr_drg,payer_category,setting,hospital_id
